In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
peft_model_id = "Ebad/bloom2-7b1-lora-tagger-for-tweet-generation"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

In [5]:
df = pd.read_csv('/content/test_df.csv', engine='python')

In [6]:
tds = Dataset.from_pandas(df)
data = DatasetDict()
data['test'] = tds

In [7]:
data['test'][0]['prediction']

'cbc with username CBCToronto posted a tweet at 2018-03-24 14:38:24 with an image that is arafed protestors hold signs in a city street with number of likes 97 that reads ->: Hundreds gather in Toronto in support of U.S. rally for gun control <hyperlink> <hyperlink>'

In [8]:
output = []

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
for i in range(0,500): #predictions
  txt = data['test'][i]['prediction']
  batch = tokenizer(txt , return_tensors='pt').to(device)
  with torch.amp.autocast('cuda'):
    output_tokens = model.generate(**batch, max_new_tokens=50)
    output.append(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [17]:
output[0]

'cbc with username CBCToronto posted a tweet at 2018-03-24 14:38:24 with an image that is arafed protestors hold signs in a city street with number of likes 97 that reads ->: Hundreds gather in Toronto in support of U.S. rally for gun control <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyper'

In [18]:
dicti = {'content':output}

In [19]:
dff = pd.DataFrame(dicti)

In [20]:
for i in range(len(dff)):
  dff.at[i,'content'] = dff.at[i,'content'].split("->: ")[1]

In [21]:
dff.to_excel('/content/output_Bloom.xlsx')

In [22]:
from google.colab import files
files.download("/content/output_Bloom.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>